In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import glob

%matplotlib inline
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'NanumGothic'

# CSV 파일들을 저장할 빈 DataFrame을 생성합니다.
stock = pd.DataFrame()

# data 폴더에 있는 모든 CSV 파일을 가져옵니다.
csv_files = glob.glob('data/*.csv')



In [13]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from itertools import combinations
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import Huber
from tensorflow.keras.models import load_model
# print(csv_files)
# 모든 CSV 파일을 순회하며 데이터를 DataFrame에 추가합니다.
# ['data\\000270_95_22기아.csv', 'data\\000660_95_22SK하이닉스.csv', 'data\\005930_95_22삼전.csv', 'data\\006400_95_22삼성SDI.csv', 'data\\007310_95_22오뚜기.csv', 'data\\012330_95_22현대모비스.csv', 'data\\035420_95_22네이버.csv', 'data\\066570_95_22LG전자.csv', 'data\\068760_95_22셀트리온제약.csv', 'data\\105560_95_22KB금융.csv', 'data\\207940_95_22삼성바이오로직스.csv']
csv_files = ['data\\207940_95_22삼성바이오로직스.csv']
WINDOW_SIZE = 20
for csv_file in csv_files:
    tmp = pd.read_csv(csv_file)
    stock = pd.concat([stock, tmp], ignore_index=True)
# print(stock)

normal = 'Open High Low Close Volume '
M1 = 'ma10'
DU = 'high_low_diff bb_upper'
M1SP = 'ma10 S&P500'
M2VM = 'ma20 volumn_ma5'
SPU = 'S&P500 bb_upper'
M1K = 'ma10 %K'
M2SP = 'ma20 S&P500'
KOU = 'Kospi bb_upper'

tmp_list = [M1, DU, M1SP, M2VM, SPU, M1K, M2SP, KOU]
result_pridict = []

for tmp in tmp_list:
    tmp_use_col = normal+tmp
    tmp = tmp.replace(" ", "_")
    use_cols = tmp_use_col.split()
    ln_use_cols =len(use_cols)
    PATH_list = ['GRU/GRU', 'LSTM/LSTM', 'LSTM_GRU/LSTM_GRU']
    for path in PATH_list:
      checkpoint_path = f'./checkpoint/{path}_{tmp}.h5'
      print(checkpoint_path, "체크포인트")

      try:
        model = load_model(checkpoint_path)

        input_scaled = stock[use_cols]
        total_data = [[input_scaled[len(input_scaled)-WINDOW_SIZE-1:len(input_scaled)-1]]]
        conform_total_data = [[input_scaled[len(input_scaled)-1:len(input_scaled)]]][:4]

        data = np.array(total_data)
        data = data.tolist()
        data = data[0]

        conform_data = conform_total_data[0][0]
        conform_data = conform_data.values[0]

        data_2d = np.array(data).reshape(-1, len(data[0][0]))

        scaler = MinMaxScaler()
        scaler.fit(data_2d)

        scaled_data_2d = scaler.transform(data_2d)

        scaled_data_3d = scaled_data_2d.reshape(len(data), len(data[0]), len(data[0][0]))

      except OSError:
          print(f"체크포인트 파일 '{checkpoint_path}'를 찾을 수 없습니다. 다음 조합으로 이동합니다.")
          continue
      # print(path, "아아아ㅏ아아아아아아아아아")
      if path == 'LSTM_GRU/LSTM_GRU':
        predicted_value = model.predict({'lstm_input': scaled_data_3d, 'gru_input': scaled_data_3d})
      else:
        predicted_value = model.predict(scaled_data_3d)
      print(predicted_value[0])
      
      predicted_value_original = scaler.inverse_transform(predicted_value)
      predicted_value_original = np.round(predicted_value_original, ln_use_cols)[0][:4]
      conform_total_data = conform_total_data[0][0].values[0]
      predicted_value_original = list(predicted_value_original)
      predicted_value_original.extend(conform_total_data[:4])
      
      use_cols_check = '/'.join(use_cols)
      predicted_value_original.append(use_cols_check)
      result_use_cols = use_cols[:4]
      result_use_cols.extend(['_Open', '_High', '_Low', '_Close', 'Use_Cols_Check'])

      result_dict = dict(zip(result_use_cols, predicted_value_original))
      result_pridict.append(result_dict)
print(result_pridict)

./checkpoint/GRU/GRU_ma10.h5 체크포인트
1/1 [==============================] - 0s 275ms/step
[0.09013563 0.10947284 0.10591173 0.06485345 0.01761834 0.10996258]
./checkpoint/LSTM/LSTM_ma10.h5 체크포인트
1/1 [==============================] - 0s 298ms/step
[0.08167008 0.05664054 0.09044835 0.07567584 0.01591187 0.08057095]
./checkpoint/LSTM_GRU/LSTM_GRU_ma10.h5 체크포인트
1/1 [==============================] - 1s 519ms/step
[0.09392332 0.08238138 0.09484047 0.09957239 0.01056364 0.08725183]
./checkpoint/GRU/GRU_high_low_diff_bb_upper.h5 체크포인트
체크포인트 파일 './checkpoint/GRU/GRU_high_low_diff_bb_upper.h5'를 찾을 수 없습니다. 다음 조합으로 이동합니다.
./checkpoint/LSTM/LSTM_high_low_diff_bb_upper.h5 체크포인트
체크포인트 파일 './checkpoint/LSTM/LSTM_high_low_diff_bb_upper.h5'를 찾을 수 없습니다. 다음 조합으로 이동합니다.
./checkpoint/LSTM_GRU/LSTM_GRU_high_low_diff_bb_upper.h5 체크포인트
체크포인트 파일 './checkpoint/LSTM_GRU/LSTM_GRU_high_low_diff_bb_upper.h5'를 찾을 수 없습니다. 다음 조합으로 이동합니다.
./checkpoint/GRU/GRU_ma10_S&P500.h5 체크포인트
1/1 [==============================] - 0

In [14]:
print(result_pridict)
import pandas as pd


# 데이터를 DataFrame으로 변환
df = pd.DataFrame(result_pridict)

# DataFrame을 엑셀 파일로 저장
df.to_excel('check3.xlsx', index=False)

[{'Open': 873318.8, 'High': 889674.4, 'Low': 848902.2, 'Close': 867307.1, '_Open': 947776.0, '_High': 947776.0, '_Low': 885513.0, '_Close': 892432.0, 'Use_Cols_Check': 'Open/High/Low/Close/Volume/ma10'}, {'Open': 872607.7, 'High': 885758.3, 'Low': 847465.7, 'Close': 868226.94, '_Open': 947776.0, '_High': 947776.0, '_Low': 885513.0, '_Close': 892432.0, 'Use_Cols_Check': 'Open/High/Low/Close/Volume/ma10'}, {'Open': 873637.06, 'High': 887666.25, 'Low': 847873.7, 'Close': 870257.94, '_Open': 947776.0, '_High': 947776.0, '_Low': 885513.0, '_Close': 892432.0, 'Use_Cols_Check': 'Open/High/Low/Close/Volume/ma10'}, {'Open': 872493.3, 'High': 885404.06, 'Low': 853362.75, 'Close': 879374.44, '_Open': 947776.0, '_High': 947776.0, '_Low': 885513.0, '_Close': 892432.0, 'Use_Cols_Check': 'Open/High/Low/Close/Volume/ma10/S&P500'}, {'Open': 874097.4, 'High': 890312.6, 'Low': 849862.8, 'Close': 870725.0, '_Open': 947776.0, '_High': 947776.0, '_Low': 885513.0, '_Close': 892432.0, 'Use_Cols_Check': 'Open/